# Funciones descarga CORDEX EUR-11
***

_Autor:_    __Salvador Navas__ <br> _Revisión:_ __10/06/2020__ <br>


__Descripción__:<br>
https://www.euro-cordex.net/060378/index.php.en

__Cosas a corregir__ <br>

In [ ]:
import xarray as xr
import tqdm
import os
import sys
import netCDF4 as nc
import numpy as np
import warnings
import datetime
import tqdm
import pandas as pd
warnings.filterwarnings('ignore')

In [ ]:
from math import *
def rotated_grid_transform(grid_in, option, SP_coor):
    """Descarga los datos de la zona de estudio del sevidor 'https://portal.nccs.nasa.gov/datashare/NEXGDDP/BCSD/' y genera los archivos NetCDF para el escenario, variable y modelo de interés
    
    Parámetros:
    -----------
    grid_in:    float. Introducir la longitud y latitud del punto que se desea rotar [lon,lat]
    option:     int. 1-->Regular -> Rotated 2-->Rotated -> Regular
    SP_coor:    float. Parámetros de rotación. Para la malla de EUR-11 se establecen (18,-39.25)
    
    Salidas:
    --------
    Vector con las coordenadas rotadas
    """
    
    
    lon = grid_in[0]
    lat = grid_in[1];

    lon = (lon*pi)/180; # Convert degrees to radians
    lat = (lat*pi)/180;

    SP_lon = SP_coor[0];
    SP_lat = SP_coor[1];
    
    #SP_lon = NP_lon - 180, SP_lat = -NP_lat.

    theta = 90+SP_lat; # Rotation around y-axis
    phi = SP_lon; # Rotation around z-axis

    theta = (theta*pi)/180;
    phi = (phi*pi)/180; # Convert degrees to radians

    x = np.cos(lon)*np.cos(lat); # Convert from spherical to cartesian coordinates
    y = np.sin(lon)*np.cos(lat);
    z = np.sin(lat);

    if option == 1: # Regular -> Rotated

        x_new = np.cos(theta)*np.cos(phi)*x + np.cos(theta)*np.sin(phi)*y + np.sin(theta)*z;
        y_new = -np.sin(phi)*x + np.cos(phi)*y;
        z_new = -np.sin(theta)*np.cos(phi)*x - np.sin(theta)*np.sin(phi)*y + np.cos(theta)*z;

    else:  # Rotated -> Regular

        phi = -phi;
        theta = -theta;

        x_new = np.cos(theta)*np.cos(phi)*x + np.sin(phi)*y + np.sin(theta)*np.cos(phi)*z;
        y_new = -np.cos(theta)*np.sin(phi)*x + np.cos(phi)*y - np.sin(theta)*np.sin(phi)*z;
        z_new = -np.sin(theta)*x + np.cos(theta)*z;



    lon_new = np.arctan2(y_new,x_new); # Convert cartesian back to spherical coordinates
    lat_new = np.arcsin(z_new);

    lon_new = (lon_new*180)/pi; # Convert radians back to degrees
    lat_new = (lat_new*180)/pi;

    return lon_new , lat_new

In [ ]:
import os
def find(path,GCM,RCM,Ensemble,Experiment,variable):
    """ Esta función permite encontrar los ficheros correspondientes a los modelos, experimentos y GCM
    Parámetros:
    -----------
    path       :    path donde se encuentran los ficheros
    GCM        :    Nombre del GCM
    RCM        :    Nombre del RCM
    Ensemble   :    Nombre del Ensemble
    Experiment :    Nombre del experimento
    variable   :    Nombre de la variable de estudio
    
    Salidas:
    --------
    Lista con los ficheros que cumplen con las caracteriísticas introducidas """
    
    directory_files=list()
    root_list=list()
    for root, dirs, files in os.walk(path):
        for file in files:
            if (variable in file) & (GCM in file) & (RCM in file) & (Ensemble in file) & (Experiment in file) & (variable in file):
                directory_files.append(file)
                root_list.append(root)
    return directory_files, np.unique(root_list)

In [2]:
def extract_CORDEX_EUR11(path_output,area=False,lon_min_area=None,lat_min_area=None,lon_max_area=None,lat_max_area=None,point=True,
                         lon_point=None,lat_point=None,name_point=None,path_G='G:/CLIMA/02_HYDRO-CLIMATE/CORDEX/'):
    """ Esta función permite descargar de G los datos de cambio climático CORDEX en el punto que introduzcamos. La propia función va a elegir el puntos más cercano.
        Además se puede extraer un area seleccionada.
        
        Parámetros:
        -----------
        path_output  :    string. path donde se encuentran los ficheros
        area         :    string. Por defecto se encuentra en None. Si se desea extraer un area de la malla poner como True
        lon_min_area :    float. Longitud minima del área. Si no se va a extraer un area poner como None
        lat_min_area :    float. Latitud minima del área. Si no se va a extraer un area poner como None
        lon_max_area :    float. Longitud máxima del área. Si no se va a extraer un area poner como None
        lat_max_area :    float. Latitud máxima del área. Si no se va a extraer un area poner como None
        point        :    string. Por defecto se encuentra en True. Si se desea extraer un area de la malla poner como None
        lon_point    :    float. Longitud del punto de estudio. Si no se va a extraer un punto poner como None
        lat_point    :    float. Latitud del punto de estudio.Si no se va a extraer un punto poner como None
        name_point   :    string. Nombre del punto en el caso de que sólo se quiera obtener datos de cambio climático  
        path_G 
        

        Salidas:
        --------
        Dataframe en formato csv con los datos de cambio climático en formato csv. En el caso de que se desee extraer, se exportará además un dataframe con las coordenadas de los puntos """
    
    
    variables=['pr','tasmax','tasmin']
    Experiment=['historical','rcp45','rcp85']
    modelos=[['CNRM-CERFACS-CNRM-CM5','SMHI-RCA4','r1i1p1'],
         ['CNRM-CERFACS-CNRM-CM5','KNMI-RACMO22E','r1i1p1'],
         ['ICHEC-EC-EARTH','SMHI-RCA4','r12i1p1'],
         ['ICHEC-EC-EARTH','KNMI-RACMO22E','r1i1p1'],
         ['ICHEC-EC-EARTH','KNMI-RACMO22E','r12i1p1'],
         ['ICHEC-EC-EARTH','DMI-HIRHAM5','r3i1p1'],
         ['ICHEC-EC-EARTH','CLMcom-CCLM4-8-17','r12i1p1'],
         ['IPSL-IPSL-CM5A-MR','SMHI-RCA4','r1i1p1'],
         ['IPSL-IPSL-CM5A-MR','IPSL-WRF381P','r1i1p1'],
         ['MOHC-HadGEM2-ES','SMHI-RCA4','r1i1p1'],
         ['MOHC-HadGEM2-ES','DMI-HIRHAM5','r1i1p1'],
         ['MOHC-HadGEM2-ES','CLMcom-CCLM4-8-17','r1i1p1'],
         ['MPI-M-MPI-ESM-LR','SMHI-RCA4','r1i1p1'],
         ['MPI-M-MPI-ESM-LR','CLMcom-CCLM4-8-17','r1i1p1'],
         ['NCC-NorESM1-M','DMI-HIRHAM5','r1i1p1']]
    p=0
    for i in tqdm.tqdm(modelos):
        for ex in Experiment:
            for v in variables:
                direc, root =find(path_G,i[0],i[1],i[2],ex,v)
                if area==True:
                    [lon_min_,lat_min_]= rotated_grid_transform((lon_min_area,lat_min_area), 1, (18,-39.25))
                    [lon_max_,lat_max_]= rotated_grid_transform((lon_max_area,lat_max_area), 1, (18,-39.25))
                    
                    if os.path.exists(path_output+v+'/'+ex+'/'+v+'_EUR-11_'+i[0]+'_'+ex+'_'+i[2]+'_'+i[1]+'_day_'+
                               direc[0][-20:-12]+'_'+direc[-1][-11:-3]+'.csv')==True:
                        continue
                    else:
                        print('### Descargando: '+v+'_EUR-11_'+i[0]+'_'+ex+'_'+i[2]+'_'+i[1]+'_day_'+direc[0][-20:-12]+'_'+direc[-1][-11:-3]+'.csv')
                        da=xr.open_mfdataset(root[0]+'/'+'*.nc')
                        da = da.sel(rlat=slice(lat_min_-1.5, lat_max_+1.5), rlon=slice(lon_min_-0.5, lon_max_+0.5))
                        [XX,YY]=np.meshgrid(da['rlon'].values,da['rlat'].values)
                        [lon_new,lat_new]=rotated_grid_transform((XX,YY), 2, (18,-39.25))
                        try:
                            time=da.indexes['time'].to_datetimeindex()
                        except:
                            time=da.indexes['time']
                        if v=='pr':
                            csv=pd.DataFrame((da[v].values*86400).flatten().reshape(len(time),np.size(XX)),
                                             index=time)
                        else:
                            csv=pd.DataFrame((da[v].values-273).flatten().reshape(len(time),np.size(XX)),
                                             index=time)
                        Coordenadas=pd.DataFrame(index=np.arange(0,len(lon_new.flatten())),columns=['Lon','Lat'])
                        Coordenadas.iloc[:,0]=lon_new.flatten()
                        Coordenadas.iloc[:,1]=lat_new.flatten()

                        csv.to_csv(path_output+v+'_EUR-11_'+i[0]+'_'+ex+'_'+i[2]+'_'+i[1]+'_day_'+
                                   direc[0][-20:-12]+'_'+direc[-1][-11:-3]+'.csv')
                        Coordenadas.to_csv(path_output+'Coordenadas'+'_EUR-11_'+i[0]+'_'+ex+'_'+i[2]+'_'+i[1]+'_day_'+
                                   direc[0][-20:-12]+'_'+direc[-1][-11:-3]+'.csv')

                        
                        if p==0:
                            fig, ax = plt.subplots(figsize=(14, 10),subplot_kw=dict(projection=ccrs.PlateCarree()))
                            cs=ax.pcolormesh(lon_new,lat_new,da[v].values[0] ,cmap=plt.get_cmap('Blues'),vmin=0)
                            ax.add_feature(cfeature.COASTLINE.with_scale('10m'))
                            ax.add_feature(cfeature.BORDERS.with_scale('10m'))
                            fig.colorbar(cs,label=v)
                            plt.legend(fontsize=15)
                            plt.plot(lon_new.flatten(),lat_new.flatten(),'or')
                        da.close()
                            
                        
                        p=p+1
                        
                elif point==True:
                    [lon_min_,lat_min_]= rotated_grid_transform((lon_point,lat_point), 1, (18,-39.25))
                    if os.path.exists(path_output+v+'/'+ex+'/'+v+'_EUR-11_'+i[0]+'_'+ex+'_'+i[2]+'_'+i[1]+'_day_'+
                               direc[0][-20:-12]+'_'+direc[-1][-11:-3]+'_'+name_point+'.csv')==True:
                        continue
                    else:
                        print('### Descargando: '+v+'_EUR-11_'+i[0]+'_'+ex+'_'+i[2]+'_'+i[1]+'_day_'+direc[0][-20:-12]+'_'+direc[-1][-11:-3]+'_'+name_point+'.csv')
                        da=xr.open_mfdataset(root[0]+'/'+'*.nc')
                        da=da.sel(rlat=lat_min_, rlon=lon_min_, method='nearest')
            
                        try:
                            time=da.indexes['time'].to_datetimeindex()
                        except:
                            time=da.indexes['time']
                        if v=='pr':
                            csv=pd.DataFrame((da[v].values*86400).flatten(),index=time)
                        else:
                            csv=pd.DataFrame((da[v].values-273).flatten(),index=time)
                        
                        csv.to_csv(path_output+v+'_EUR-11_'+i[0]+'_'+ex+'_'+i[2]+'_'+i[1]+'_day_'+
                                   direc[0][-20:-12]+'_'+direc[-1][-11:-3]+'_'+name_point+'.csv')

                        da.close()